# Chains - Encadeamento com Langchain
___

In [8]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass(
        prompt="Digite sua chave de API do OpenAI: "
    )

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [9]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Crie uma frase sobre o seguinte tema: {assunto}")

In [10]:
chain = prompt | model

In [11]:
response = chain.invoke({"assunto": "inteligência artificial"})

In [12]:
print(response.content)

A inteligência artificial está revolucionando o mundo, tornando possíveis feitos que antes só eram imaginados em filmes de ficção científica.


In [13]:
prompt = ChatPromptTemplate.from_template("Crie uma frase sobre o seguinte tema: {assunto}")
chain = prompt | model

`Remover os metadados sem o uso do (print(response.content))`

In [14]:
from langchain_core.output_parsers import StrOutputParser
chain = prompt | model | StrOutputParser()
response = chain.invoke({"assunto": "gatinhos"})
print(chain.invoke({"assunto": "gatinhos"}))

Os gatinhos são seres adoráveis e cheios de personalidade, capazes de trazer alegria e amor para nossas vidas.


![alt text](image.png)

Fluxo de Execução:
1. Prompt Template (`prompt`)
    * Função: Formata a entrada do usuário em um prompt estruturado
    * Input: Variáveis/texto bruto
    * Output: Prompt formatado para o modelo

2. Language Model (`model`)
    * Função: Processa o prompt e gera resposta
    * Input: Prompt formatado
    * Output: Objeto de resposta do modelo (com metadados)
3. Output Parser (StrOutputParser())
    * Função: Extrai apenas o texto da resposta
    * Input: Objeto completo do modelo
    * Output: String limpa
Operador Pipe (`|`)
**LCEL**: LangChain Expression Language
Conecta: Componentes sequencialmente
Passa: Output de um como input do próximo

In [16]:
import json
print(json.dumps(prompt.input_schema.model_json_schema(), indent=4))

{
    "properties": {
        "assunto": {
            "title": "Assunto",
            "type": "string"
        }
    },
    "required": [
        "assunto"
    ],
    "title": "PromptInput",
    "type": "object"
}
